### 비정형 데이터 융합형 기업 예측 모델
* 뉴스나 전문가 의견및 여론 기반한 비정형 데이터 수집
* 주가 등 정형 데이터 수집
* 기업의 주가 영향 요소 예측등을 통한
* 기업의 위험관리및 투자예측 분석

설립일:1955년 12월 9일 (65주년)[2]
주소
서울특별시 서초구 신반포로 194 (반포동)[3]
업종명
백화점
상장유무
상장기업
기업규모
대기업
상장시장
유가증권시장 (1985년 ~ 현재)
대표이사
차정호
종목코드
004170
자본금
492억 2천만원 (2019)
매출액
1조 5,576억원 (2019)
영업이익
2,220억 9천만원 (2019)
당기순이익
7,059억 5천만원 (2019)
직원 수
2,745명 (2020.06.30)

### 분석요소
* 뉴스 :
* SNS  :
* 댓글 :
* 재무재표: 
* 기업분석 지표:

* 제품 매출 현황

* 시가 :
* 종가 :
* 고가 :
* 저가 :
* 거래량:

### 네이버 뉴스 수집
* 신세계
* url = https://search.naver.com/search.naver?query=%EC%8B%A0%EC%84%B8%EA%B3%84&where=news&ie=utf8&sm=nws_hty

In [1]:
url='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'.format('신세계',11)
url

'https://search.naver.com/search.naver?&where=news&query=신세계&start=11'

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [3]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [4]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [5]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [6]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [7]:
def insMain(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into nnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [8]:
def getPage(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    tgts=htmls.select('ul.list_news > li')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [11]:
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'
for i in range(1000,2000):
    url=urlsrc.format('신세계',i*10+1)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=신세계&start=10001
https://search.naver.com/search.naver?&where=news&query=신세계&start=10001
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=10011
https://search.naver.com/search.naver?&where=news&query=신세계&start=10011
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아

https://search.naver.com/search.naver?&where=news&query=신세계&start=10141
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=10151
https://search.naver.com/search.naver?&where=news&query=신세계&start=10151
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기

https://search.naver.com/search.naver?&where=news&query=신세계&start=10281
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=10291
https://search.naver.com/search.naver?&where=news&query=신세계&start=10291
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기

https://search.naver.com/search.naver?&where=news&query=신세계&start=10421
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=10431
https://search.naver.com/search.naver?&where=news&query=신세계&start=10431
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기

https://search.naver.com/search.naver?&where=news&query=신세계&start=10561
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=10571
https://search.naver.com/search.naver?&where=news&query=신세계&start=10571
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.:신세계조선호텔  신세계  뗀다 영문명도  JOSUN  변경
이미 존재 합니다.: 신세계 프로필  유신열 신세계디에프 대표
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기

...

https://search.naver.com/search.naver?&where=news&query=신세계&start=19431
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.: 코로나 쇼크  신세계  지난해 영업익 883억  전년비 81  급감
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=19441
https://search.naver.com/search.naver?&where=news&query=신세계&start=19441
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 

https://search.naver.com/search.naver?&where=news&query=신세계&start=19551
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.: 코로나 쇼크  신세계  지난해 영업익 883억  전년비 81  급감
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=19561
https://search.naver.com/search.naver?&where=news&query=신세계&start=19561
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 

https://search.naver.com/search.naver?&where=news&query=신세계&start=19671
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.: 코로나 쇼크  신세계  지난해 영업익 883억  전년비 81  급감
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=19681
https://search.naver.com/search.naver?&where=news&query=신세계&start=19681
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 

https://search.naver.com/search.naver?&where=news&query=신세계&start=19791
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.: 코로나 쇼크  신세계  지난해 영업익 883억  전년비 81  급감
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=19801
https://search.naver.com/search.naver?&where=news&query=신세계&start=19801
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 

https://search.naver.com/search.naver?&where=news&query=신세계&start=19911
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 상품권 받아가세요 
이미 존재 합니다.:김래원X이다희  루카  더 비기닝  추격 액션 신세계
이미 존재 합니다.: 홈술  유행 속 와인 인기  신세계百  작년 판매 41   
이미 존재 합니다.: 호텔 살려라    이마트  신세계조선호텔에 2 700억원 추가 수혈
이미 존재 합니다.:신세계 인수 충격  SK는 왜 돌연 야구단을 파는 걸까   MK이슈 
이미 존재 합니다.: 코로나 쇼크  신세계  지난해 영업익 883억  전년비 81  급감
이미 존재 합니다.:신세계인터   바나나 리퍼블릭  한국 공식 런칭  합리적 럭셔리족 공략 
https://search.naver.com/search.naver?&where=news&query=신세계&start=19921
https://search.naver.com/search.naver?&where=news&query=신세계&start=19921
이미 존재 합니다.: 정장수요 감소에도 웃었다  신세계인터내셔날 텐먼스  지난해 매출 270  초   
이미 존재 합니다.:신세계그룹  SK텔레콤과  SK와이번스  야구단 인수 합의
이미 존재 합니다.: MD포토  추신수  신세계 야구단 1호 영입 
이미 존재 합니다.: MD포토  추신수  인천 신세계로 합류 
이미 존재 합니다.: MD포토  신세계 추신수를 향한 취재열기
이미 존재 합니다.: 우체국서 U 알뜰폰 가입하고 신세계 